In [1]:
import sys
sys.path.insert(0,'/kaggle/input/llm-packages')
batch_size = 6
TARGET_MODEL = "/kaggle/input/mistral-7b-v0-1/Mistral-7B-v0.1"

device = 'cuda'
DEBUG = False

In [2]:
import pandas as pd
import numpy as np
from functools import partial
INPUT_DIR = "../input/"
test_df = pd.read_csv(INPUT_DIR + "llm-detect-ai-generated-text/test_essays.csv", sep=',')

test_df.shape: (3, 3)


In [3]:
!pip install -q -U accelerate --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/

In [4]:
from transformers import AutoTokenizer, MistralForSequenceClassification
from llm_util import *
tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
from transformers import BitsAndBytesConfig
from peft import PeftType,PromptEncoderConfig,get_peft_model

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
model = MistralForSequenceClassification.from_pretrained(TARGET_MODEL,num_labels=2,quantization_config=nf4_config, \
                                                          device_map={"":0})
peft_type = PeftType.P_TUNING
peft_config = PromptEncoderConfig(task_type="SEQ_CLS", num_virtual_tokens=8, \
                                  encoder_hidden_size=4096,encoder_dropout=0.1,\
                                  encoder_num_layers=2,encoder_reparameterization_type='MLP')#'LSTM')
model = get_peft_model(model, peft_config)
model.config.pad_token_id = tokenizer.pad_token_id
model.load_state_dict(torch.load('/kaggle/input/llm-model/weights.pth'),strict=False);
model = model.half()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/mistral-7b-v0-1/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


_IncompatibleKeys(missing_keys=['base_model.model.embed_tokens.weight', 'base_model.model.layers.0.self_attn.q_proj.weight', 'base_model.model.layers.0.self_attn.k_proj.weight', 'base_model.model.layers.0.self_attn.v_proj.weight', 'base_model.model.layers.0.self_attn.o_proj.weight', 'base_model.model.layers.0.mlp.gate_proj.weight', 'base_model.model.layers.0.mlp.up_proj.weight', 'base_model.model.layers.0.mlp.down_proj.weight', 'base_model.model.layers.0.input_layernorm.weight', 'base_model.model.layers.0.post_attention_layernorm.weight', 'base_model.model.layers.1.self_attn.q_proj.weight', 'base_model.model.layers.1.self_attn.k_proj.weight', 'base_model.model.layers.1.self_attn.v_proj.weight', 'base_model.model.layers.1.self_attn.o_proj.weight', 'base_model.model.layers.1.mlp.gate_proj.weight', 'base_model.model.layers.1.mlp.up_proj.weight', 'base_model.model.layers.1.mlp.down_proj.weight', 'base_model.model.layers.1.input_layernorm.weight', 'base_model.model.layers.1.post_attention_l

In [17]:
val_data = InfData(test_df)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, \
                          collate_fn=partial(collate_inf,tokenizer=tokenizer))

In [19]:
probs = []
for input_ids,attention_mask in val_loader:
    input_ids,attention_mask = input_ids.to('cuda'),attention_mask.to('cuda')
    with torch.no_grad():
        out = model(input_ids=input_ids,attention_mask=attention_mask).logits[:,0].detach().cpu().numpy()
    probs.append(out)
probs = np.concatenate(probs)

In [21]:
sub = pd.DataFrame()
sub['id'] = test_df['id']
sub['generated'] = probs
sub.to_csv('submission.csv', index=False)

,id,generated
0,0000aaaa,-1.176758
1,1111bbbb,-3.904297
2,2222cccc,-3.126953
